# <center>L-BFGS метода оптимизације</center>

L-BFGS (Limited-memory BFGS) је квази-Њутнова метода оптимизације другог реда заснована на методи BFGS (Broyden–Fletcher–Goldfarb–Shanno). Идеја L-BFGS метода је да се унапреди стандардна BFGS метода уклањањем потребе за складиштењем целокупне апроксимације инверза хесијана, уместо које се чува одређени број последњих разлика градијената и разлика решења из претходних корака, на основу којих се може ефикасним поступком апроксимирати производ $H_k^{-1}\nabla f_k$. Наиме, BFGS метод оптимизације има следеће правило ажурирања текућег решења:

$$x_{k+1} = x_{k} - \alpha_k H_k^{-1} \nabla f_k \tag{1}$$

при чему су $\alpha_k$ корак, а $H_k^{-1}$ апроксимација инверза хесијана у $k$-тоj итерацији. $H_k^{-1}$ се ажурира према следећем правилу:

$$H_{k+1}^{-1} = V_k^TH_k^{-1}V_k + \rho_ks_ks_k^T \tag{2}$$

где су:

$$\rho_k = \frac{1}{y_k^Ts_k},\hspace{3em}V_k = I - \rho_ky_ks_k^T, \tag{3}$$

$$s_k = x_{k+1} - x_k,\hspace{3em}y_k = \nabla f_{k+1} - \nabla f_k. \tag{4}$$

Да би се избегло чување апроксимације инверза хесијана $H_k^{-1}$, која је у општем случају густа матрица, прибегава се чувању модификоване верзије $H_k^{-1}$ имплицитно, чувањем $m$ парова вектора $\{s_i, y_i\}$ добијених горепоменутим изразима (2)-(4) у последњих $m$ итерација. На основу вектора $\{s_i, y_i\}$ и $\nabla f_k$, израчунава се производ $H_k^{-1}\nabla f_k$ поступком који ће у наставку бити описан.

Наиме, у $k$-тој итерацији, текуће решење је $x_k$, и скуп парова вектора $\{s_i, y_i\}$ има вредности за $i = k - m, ... , k - 1$. Потребно је одабрати неку иницијалну апроксимацију инверза хесијана $H_k^0$, на основу које, узастопном применом правила (2), се добија следећи израз:

\begin{equation}
\begin{split}
H_k^{-1} & = (V_{k-1}^T\cdots V_{k-m}^T)H_k^0(V_{k-m}\cdots V_{k-1})
\\&+ \rho_{k-m}(V_{k-1}^T\cdots V_{k-m+1}^T)s_{k-m}s_{k-m}^T(V_{k-m+1}\cdots V_{k-1})
\\&+ \rho_{k-m+1}(V_{k-1}^T\cdots V_{k-m+2}^T)s_{k-m+1}s_{k-m+1}^T(V_{k-m+2}\cdots V_{k-1})
\\&+ \cdots
\\&+ \rho_{k-1}s_{k-1}s_{k-1}^T
\end{split}
\tag{5}
\end{equation}

На основу израза (5), уз нешто алгебарског сређивања, природно произилази следећи алгоритам за рачунање $H_k^{-1}\nabla f_k$:

$$
\begin{align*}
&Улаз:\hspace{1ex}текуће\hspace{1ex}решење\hspace{1ex}x_k,\hspace{1ex}дужина\hspace{1ex}''историје''\hspace{1ex}m \\
&Излаз:\hspace{1ex}вредност\hspace{1ex}производа\hspace{1ex}H_k^{-1}\nabla f_k \\
&q \leftarrow \nabla f_k;\\
&\textbf{for}\hspace{1ex}i = k-1,k-2,...,k-m \\
&\hspace{3em}\alpha_i \leftarrow \rho_is_i^Tq; \\
&\hspace{3em}q \leftarrow q - \alpha_iy_i; \\
&\textbf{end}\hspace{1ex}(\textbf{for}) \\
&r \leftarrow H_k^0q; \\
&\textbf{for}\hspace{1ex}i=k-m,k-m+1,...,k-1 \\
&\hspace{3em}\beta \leftarrow \rho_iy_i^Tr; \\
&\hspace{3em}r \leftarrow r + s_i(\alpha_i - \beta); \\
&\textbf{end}\hspace{1ex}(\textbf{for}) \\
\end{align*}
\tag{A1}
$$

Треба још одлучити како се бира иницијална апроксимација хесијана $H_k^0$. Релативно једноставан метод који у пракси даје добре резултате је да се узима:

$$H_k^0 = \gamma_kI,\hspace{3em}\gamma_k = \frac{s_{k-1}^Ty_{k-1}}{y_{k-1}^Ty_{k-1}} \tag{6}$$

Као и за стандардну BFGS методу, корак $\alpha_k$ из правила ажурирања (1) бира се тако да задовољава (6) Вулф услове (енг. *Wolfe conditions*) или (7) строге Вулф услове (енг. *strong Wolfe conditions*):

$$
\begin{align*}
f(x_k+\alpha_kp_k) &\leq f(x_k)+c_1\alpha_k\nabla f_k^Tp_k \\
\nabla f(x_k+\alpha_kp_k)^Tp_k &\geq c_2\nabla f_k^Tp_k \\
\end{align*}
\tag{6}
$$

$$
\begin{align*}
f(x_k+\alpha_kp_k) &\leq f(x_k)+c_1\alpha_k\nabla f_k^Tp_k \\
\nabla |f(x_k+\alpha_kp_k)^Tp_k| &\leq c_2|\nabla f_k^Tp_k| \\
\end{align*}
\tag{7}
$$

где су $0 < c_1 < c_2 < 1$. У пракси се често узимају вредности $c_1=10^{-4}$ и $c_2=0.9$. Први услов у (6) и (7) је тзв. Армихо услов (енг. *Armijo condition*) који гарантује пад вредности циљне функције довољан за конвергенцију, док је други услов тзв. услов кривине (енг. *curvature condition*) који елими\-нише премале кораке. Специјално, код строгих Вулф услова, елиминишу се тачке које нису у широј околини локалног минимума или стационарне тачке.

Алгоритам за налажење корака $\alpha_k$ који испуњава строге Вулф услове извршава се у два корака: прво се проналази интервал који садржи прихватљиве вредности корака, а затим се (алгоритмом *zoom*) проналази задовољавајућа вредност корака у оквиру тог интервала.

$$
\begin{align*}
&Улаз:\hspace{1ex}функција\hspace{1ex}\phi(\alpha),\hspace{1ex}параметри\hspace{1ex}c_1\hspace{1ex}и\hspace{1ex}c_2 \\
&Излаз:\hspace{1ex}корак\hspace{1ex}\alpha_*\hspace{1ex}који\hspace{1ex}испуњава\hspace{1ex}строге\hspace{1ex}Вулф\hspace{1ex}услове\hspace{1ex}(7) \\
&\alpha_0 \leftarrow 0 \\
&одабрати\hspace{1ex}\alpha_{max} > 0\hspace{1ex}и\hspace{1ex}\alpha_1 \in (0,\alpha_{max}); \\
&i \leftarrow 1; \\
&\textbf{repeat} \\
&\hspace{3em}израчунати\hspace{1ex}\phi(\alpha_i); \\
&\hspace{3em}\textbf{if}\hspace{1ex}\phi(\alpha_i) > \phi(0) + c_1\alpha_i\phi'(0)\hspace{1ex}or\hspace{1ex}[\phi(\alpha_i) \geq \phi(\alpha_{i-1})\hspace{1ex}and\hspace{1ex}i > 1] \\
&\hspace{6em}\alpha_* \leftarrow \textbf{zoom}(\alpha_{i-1},\alpha_{i}); \\
&\hspace{6em}\textbf{stop}\\
&\hspace{3em}израчунати\hspace{1ex}\phi'(\alpha_i); \\
&\hspace{3em}\textbf{if}\hspace{1ex}|\phi'(\alpha_i)| \leq -c_2\phi'(0) \\
&\hspace{6em}\alpha_* \leftarrow \alpha_{i}; \\
&\hspace{6em}\textbf{stop}\\
&\hspace{3em}\textbf{if}\hspace{1ex}\phi'(\alpha_i) \geq 0 \\
&\hspace{6em}\alpha_* \leftarrow \textbf{zoom}(\alpha_{i},\alpha_{i-1}); \\
&\hspace{6em}\textbf{stop}\\
&\hspace{3em}одабрати\hspace{1ex}\alpha_{i+1} \in (\alpha_i,\alpha_{max}); \\
&\hspace{3em}i \leftarrow i + 1; \\
&\textbf{end}\hspace{1ex}(\textbf{repeat})
\end{align*}
\tag{A2}
$$

где је $\phi(\alpha) = f(x_k + \alpha p_k)$. Следи опис алгоритма $\textbf{zoom}(\alpha_{lo}, \alpha_{hi})$ коришћеног у алгоритму линијске претраге (А2):

$$
\begin{align*}
&Улаз:\hspace{1ex}функција\hspace{1ex}\phi(\alpha),\hspace{1ex}границе\hspace{1ex}интервала\hspace{1ex}\alpha_{lo}\hspace{1ex}и\hspace{1ex}\alpha_{hi}\\
&Излаз:\hspace{1ex}задовољавајући\hspace{1ex}корак\hspace{1ex}\alpha_*\hspace{1ex}из\hspace{1ex}интервала\hspace{1ex}[\alpha_{lo},\alpha_{hi}]\\
&\textbf{repeat} \\
&\hspace{3em}некако\hspace{1ex}одабрати\hspace{1ex}корак\hspace{1ex}\alpha_j\hspace{1ex}између\hspace{1ex}\alpha_{lo}\hspace{1ex}и\hspace{1ex}\alpha_{hi}\hspace{1ex}(на\hspace{1ex}пример,\hspace{1ex}бисекцијом); \\
&\hspace{3em}израчунати\hspace{1ex}\phi(\alpha_j); \\
&\hspace{3em}\textbf{if}\hspace{1ex}\phi(\alpha_j)>\phi(0)+c_1\alpha_j\phi'(0)\hspace{1ex}or\hspace{1ex}\phi(\alpha_j) \geq \phi(\alpha_{lo}) \\
&\hspace{6em}\alpha_{hi} \leftarrow \alpha_j; \\
&\hspace{3em}\textbf{else} \\
&\hspace{6em}израчунати\hspace{1ex}\phi'(\alpha_j); \\
&\hspace{6em}\textbf{if}\hspace{1ex}|\phi'(\alpha_j)|\leq -c_2\phi'(0) \\
&\hspace{9em}\alpha_* \leftarrow \alpha_j; \\
&\hspace{9em}\textbf{stop} \\
&\hspace{6em}\textbf{if}\hspace{1ex}\phi'(\alpha_j)(\alpha_{hi}-\alpha_{lo}) \geq 0 \\
&\hspace{9em}\alpha_{hi} \leftarrow \alpha_{lo}; \\
&\hspace{6em}\alpha_{lo} \leftarrow \alpha_{j}; \\
&\textbf{end}\hspace{1ex}(\textbf{repeat}) \\
\end{align*}
\tag{A3}
$$

Пошто су алгоритмима (А1)-(А3) дати сви потребни помоћни алгоритми, сада се коначно може дати и сам алгоритам L-BFGS:

$$
\begin{align*}
&Улаз:\hspace{1ex}циљна\hspace{1ex}функција\hspace{1ex}f,\hspace{1ex}њен\hspace{1ex}градијент\hspace{1ex}\nabla f,\hspace{1ex}почетна\hspace{1ex}тачка\hspace{1ex}x_0,\hspace{1ex}дужина\hspace{1ex}''историје''\hspace{1ex}m,\hspace{1ex}параметар\hspace{1ex}конвергенције\hspace{1ex}\epsilon \\
&Излаз:\hspace{1ex}апроксимација\hspace{1ex}минимума\hspace{1ex}x^* \\
&k \leftarrow 0; \\
&\textbf{repeat} \\
&\hspace{3em}израчунати\hspace{1ex}H_k^0\hspace{1ex}по\hspace{1ex}правилу\hspace{1ex}(6); \\
&\hspace{3em}израчунати\hspace{1ex}p_k \leftarrow -H_k^{-1}\nabla f_k\hspace{1ex}алгоритмом\hspace{1ex}(А1); \\
&\hspace{3em}израчунати\hspace{1ex}x_{k+1} \leftarrow x_k + \alpha_k p_k\hspace{1ex}где\hspace{1ex}је\hspace{1ex}\alpha_k\hspace{1ex}добијено\hspace{1ex}применом\hspace{1ex}алгоритама\hspace{1ex}(А2)\hspace{1ex}и\hspace{1ex}(А3); \\
&\hspace{3em}\textbf{if}\hspace{1ex}||\nabla f_{k+1} - \nabla f_k|| < \epsilon \\
&\hspace{6em}x^* \leftarrow x_{k+1}; \\
&\hspace{6em}\textbf{stop} \\
&\hspace{3em}\textbf{if}\hspace{1ex}k > m \\
&\hspace{6em}обрисати\hspace{1ex}пар\hspace{1ex}вектора\hspace{1ex}\{s_{k-m},y_{k-m}\}\hspace{1ex}из\hspace{1ex}меморије; \\
&\hspace{3em}израчунати\hspace{1ex}s_k \leftarrow x_{k+1}-x_k, y_k = \nabla f_{k+1} - \nabla f_k; \\
&\hspace{3em}k \leftarrow k + 1; \\
&\textbf{end}\hspace{1ex}(\textbf{repeat}) \\
\end{align*}
\tag{A4}
$$

## <center>Имплементација L-BFGS метода оптимизацијe <br/> у програмском језику Python</center>

In [2]:
import numpy as np
from scipy import optimize

Алгоритам (А2) имплементира следећа Python функција:

In [3]:
def wolfe_line_search(f, grad, x, p, max_iter=100, c1=10**-4, c2=0.9, alpha_1=1.0, alpha_max=1000):
        
    def phi(alpha):
        return f(x + alpha * p)

    def phi_grad(alpha):
        return np.dot(grad(x + alpha * p).T, p)

    alpha_i_1 = 0
    alpha_i = alpha_1

    for i in range(1, max_iter + 1):
        phi_alpha_i = phi(alpha_i)
        if (phi_alpha_i > phi(0) + c1 * alpha_i * phi_grad(0)) or (i > 1 and phi_alpha_i >= phi(alpha_i_1)):
            return zoom(phi, phi_grad, alpha_i_1, alpha_i, c1, c2)
        
        phi_grad_alpha_i = phi_grad(alpha_i)
        if np.abs(phi_grad_alpha_i) <= -c2 * phi_grad(0):
            return alpha_i
        if phi_grad_alpha_i >= 0:
            return zoom(phi, phi_grad, alpha_i, alpha_i_1, c1, c2)
        alpha_i_1 = alpha_i
        alpha_i = min(2 * alpha_i, alpha_max)

    if i == max_iter:
        return None

Осим дефинисања функције $\phi$ и $\phi'$ преко прослеђене циљне функције и њеног градијента, функција `wolfe_line_search` прати алгоритам (А2) готово линију по линију. Битно је приметити да се за $\alpha_1$ увек узима вредност 1 која ће сигурно бити у интервалу $(0, \alpha_{max})$, као и да се за наредни корак $\alpha_{i+1}$ узима дупло већа вредност (уколико не премашује $\alpha_{max}$).

Следећа Python функција имплементира помоћну процедуру `zoom` дату алгоритмом (А3):

In [4]:
def zoom(phi, phi_grad, alpha_lo, alpha_hi, c1, c2, max_iter=100):
    i = 0
    while True:
        alpha_j = (alpha_lo + alpha_hi) / 2.0
        phi_alpha_j = phi(alpha_j)

        if (phi_alpha_j > phi(0) + c1 * alpha_j * phi_grad(0)) or (phi_alpha_j >= phi(alpha_lo)):
            alpha_hi = alpha_j
        else:
            phi_grad_alpha_j = phi_grad(alpha_j)
            if np.abs(phi_grad_alpha_j) <= -c2 * phi_grad(0):
                return alpha_j
            if phi_grad_alpha_j * (alpha_hi - alpha_lo) >= 0:
                alpha_hi = alpha_lo
            alpha_lo = alpha_j
        i += 1
        if i >= max_iter:
            return None

Функција `zoom` такође прати алгоритам (А3) линију по линију, једина имплементациона одлука је бирање $\alpha_j$ простом бисекцијом интервала. 

Коначно, Python функција `lbfgs` имплементира L-BFGS метод оптимизације описан алгоритмом (А4):

In [5]:
def lbfgs(f, grad, x0, eps=1e-4, max_iter=1000, history=10):
    
    n = len(x0)
    
    def two_loop_rec(x, m):
        q = grad(x)
        k = S.shape[0]
        rhos = np.zeros(k)
        alphas = np.zeros(k)
        beta = 0
        for i in range(k - 1, -1, -1):
            rhos[i] = np.dot(Y[i].T, S[i]) ** -1
            alphas[i] = np.dot(S[i].T, q) * rhos[i]
            q = q - alphas[i] * Y[i]
        if k > 0:
            gamma_k = np.dot(S[k - 1].T, Y[k - 1]) / np.dot(Y[k - 1], Y[k - 1])
            H_k0 = np.diag(gamma_k * np.ones(n))
        else:
            H_k0 = np.diag(np.ones(n))
        r = np.dot(H_k0, q)
        for i in range(k):
            beta = rhos[i] * np.dot(Y[i].T, r)
            r = r + S[i] * (alphas[i] - beta)
        
        return r
    
    S = np.empty([0, n])
    Y = np.empty([0, n])
    x_old = x0
    
    for k in range(1, max_iter + 1):
        p_k = -two_loop_rec(x_old, history)
        alpha_k = wolfe_line_search(f, grad, x_old, p_k)
        
        if alpha_k is None:
            print("Wolfe line search did not converge")
            return x_old, k
        
        x_new = x_old + alpha_k * p_k
        
        grad_diff = grad(x_new) - grad(x_old)
        if np.linalg.norm(grad_diff) < eps:
            break
        
        if k > history:
            S = S[1:]
            Y = Y[1:]
        S = np.append(S, [x_new - x_old], axis=0)
        Y = np.append(Y, [grad_diff], axis=0)
        x_old = x_new
        
    if k == max_iter:
        print("Optimization did not converge")
    else:
        print("Optimization converged in {} steps".format(k))
        
    return x_new, k

Функција `two_loop_rec` имплементира поступак из алгоритма (А1) за рачунање производа $H_k^{-1}\nabla f_k$, док низови $S$ и $Y$ представљају $\{s_i,y_i\}$, то јест, разлике решења и градијената из претходних корака.

Сада ће бити демонстрирано коришћење имплементиране методе оптимизације на функцији $f(x_0, x_1) = 2x_0^2 + 3x_1^2 +4\sin{x_0}$

In [6]:
def f(x):
    return 2 * x[0]**2 + 3 * x[1]**2 + 4 * np.sin(x[0])

In [7]:
def grad_f(x):
    return np.array([4 * x[0] + 4 * np.cos(x[0]), 6 * x[1]])

In [8]:
lbfgs(f, grad_f, np.array([1, 1]))

Optimization converged in 7 steps


(array([-7.39085133e-01,  1.45057323e-10]), 7)

Провера добијеног резултата коришћењем имплементације из библиотеке `scipy`: 

In [9]:
optimize.minimize(f, np.array([1,1]), method='L-BFGS-B', jac=grad_f, options={'maxcor':10, 'gtol':1e-4})

      fun: -1.6019544484516701
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([4.82010287e-06, 1.14993299e-06])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 8
      nit: 6
     njev: 8
   status: 0
  success: True
        x: array([-7.39084413e-01,  1.91655499e-07])

## <center>Тестирање ове имплементације метода L-BFGS <br/> над неким проблемима из скупа CUTEst</center>

CUTEst (Constrained and Unconstrained Testing Enviorment with safe threads) представља трећу по реду (после CUTEr) итерацију оргиналног CUTE окружења за тестирање оптимизационог софтвера. Оно садржи значајну количину разноврсних проблема који се користе за тестирање исправности имплементације и анализу перформанси оптимизационих алгоритама. У овом делу рада ће ова горепредстављена имплементација L-BFGS методе оптимизације бити упоређена са библиотичком имплементацијом из библиотеке `scipy`, као и са имплементацијом класичне Њутнове методе из исте библиотеке над неким од проблема из скупа CUTEst.

За програмски језик Python постоји интерфејс који омогућава коришћење CUTEst проблема који су иначе везани за програмске језике Fortran и MATLAB. Тај интерфејс имплементира библиотека `pycutest`. Нажалост, због комплексног процеса инсталације ове библиотеке, није могуће једноставно инсталирати ову библиотеку у оквиру Anaconda окружења. Због тога, поменуто тестирање биће изведено у оквиру датотеке `cutest_tests.py` која се може пронаћи у оквиру овог репозиторијума, а овде ће резултати бити приказани и прокоментарисани. Резултати овог тестирања могу се поновити једноставним покретањем поменутог скрипта на рачунару са правилно инсталираном библиотеком `pycutest`.

У следећој табели могу се видети називи и број променљивих у оквиру проблема који ће бити коришћени у тестирању:

|Назив проблема|Број променљивих|
|---|---|
|BOX|10000|
|DIXMAANL|1500|
|EIGENALS|110|
|FREUROTH|1000|
|TRIDIA|1000|
|VAREIGVL|5000|